# Project Description

*   Lending Club
*   BUAN 5510 - Capstone Project
*   Team 4 (Team Lee) - Ki Min Lee, Sylvester Setio, Thinh Mai, Yovinda Pricila
*   Professor Ben Kim



## VI. Importing Library

In [46]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.cluster import KMeans
from sklearn import metrics
from sklearn.preprocessing import StandardScaler
from scipy.cluster import hierarchy
from sklearn.cluster import AgglomerativeClustering
from matplotlib import pyplot
import seaborn as sns

from sklearn.decomposition import PCA
from sklearn.preprocessing import scale
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
# from sklearn.ensemble import RandomForestRegressor
# from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from sklearn.datasets import make_classification
from scipy import stats
from matplotlib import pyplot as plt


# Opening File and Data Preprocessing

I. Open the loan.csv file and cherrypick certain variables

In [47]:
df = pd.read_csv("loan.csv", low_memory = False)

In [48]:
df.shape

(2260668, 145)

## II. Change certain data types

In [49]:
# Change certain columns to date time
df['earliest_cr_line_year'] = df['earliest_cr_line'].str.strip().str[-4:].fillna(0).astype('int')
df['earliest_cr_line'] = pd.to_datetime(df['earliest_cr_line'])
df['issue_d'] = pd.to_datetime(df['issue_d'])

# Create new column in loan data of previous quarter
df['pqissue_d'] = df['issue_d'] - pd.tseries.offsets.DateOffset(months = 3)
df['issue_q'] = pd.PeriodIndex(pd.to_datetime(df['issue_d']), freq = 'Q')
df['pqissue_q'] = pd.PeriodIndex(pd.to_datetime(df['pqissue_d']), freq = 'Q')

# Check the distribution of age of credit when applying
df['earliest_cr_line'] = pd.to_datetime(df['earliest_cr_line'])
df['issue_d'] = pd.to_datetime(df['issue_d'])

We add one column to convert earliest_cr_line (month-year format) to earliest_cr_line_year (year format). We have converted earliest_cr_line and issue_d to date time format as well.

We created new columns pqissue_d - previous quarter issue date - it's issue date one quarter before, issue_q - quarter of the issue date, and pqissue_q - previous quarter of issue q. This will be used for joining with other data sets.

Add 4 columns: earliest_cr_line_year, pqissue_d, issue_q, and pqissue_q

In [50]:
df.shape

(2260668, 149)

In [51]:
# Change last payment date to datetime format
df['last_pymnt_d'] = pd.to_datetime(df['last_pymnt_d'])

convert to datetime format for last_pymnt_d
The calculation at the bottom is for number of successive payments. But, it must be compared with another variable like how far along is the loan, thus we don't think this last_pymnt_d will be useful.

In [52]:
((df['last_pymnt_d']-df['issue_d']))/np.timedelta64(1,'D')

0           62.0
1           62.0
2           62.0
3           62.0
4           62.0
           ...  
2260663    488.0
2260664    304.0
2260665    488.0
2260666    488.0
2260667    488.0
Length: 2260668, dtype: float64

In [53]:
# Create new column for rate of return formula
df['NAR'] = ((df['total_pymnt']/df['funded_amnt'])**(1/(365-((df['last_pymnt_d']-df['issue_d'])/np.timedelta64(1,'D'))))-1)*100
df[['NAR', 'total_pymnt', 'funded_amnt', 'last_pymnt_d', 'issue_d']].head(5)

,NAR,total_pymnt,funded_amnt,last_pymnt_d,issue_d
0,-0.889071,167.02,2500,2019-02-01,2018-12-01
1,-0.982274,1507.11,30000,2019-02-01,2018-12-01
2,-0.870187,353.89,5000,2019-02-01,2018-12-01
3,-0.866056,286.71,4000,2019-02-01,2018-12-01
4,-1.000990,1423.21,30000,2019-02-01,2018-12-01


We are keeping total_pymnt for the NAR calculation, but we will not be using NAR in our first model

In [54]:
df.shape

(2260668, 150)

### Data Preprocessing for Outlier Analysis and Feature Selection

Transforming 10+ years to 10 years and less than 1 year to 0 years so it could show in order from smallest to largest on the bar graph.

emp_length is a string, so we're converting it into numbers

In [55]:
df['emp_length'].replace(to_replace='10+ years', value='10 years', inplace=True)
df['emp_length'].replace('< 1 year', '0 years', inplace=True)


def emp_length_to_int(s):
    if pd.isnull(s):
        return s
    else:
        return np.int8(s.split()[0])

df['emp_length'] = df['emp_length'].apply(emp_length_to_int)


Logging different items for easier visualizations
We are adding 2 columns for logging

In [56]:
df['log_annual_inc'] = df['annual_inc'].apply(lambda x: np.log10(x+1))
df['log_revol_bal'] = df['revol_bal'].apply(lambda x: np.log10(x+1))


Calculate age of credit

In [57]:
df['ageOfCredit'] = ((df['issue_d'] - df['earliest_cr_line'])/np.timedelta64(1, 'Y'))

In [58]:
df.shape

(2260668, 153)

## III. Open csv file converting abbreviated state name into full state name
We got the abbreviated state name from this website
Resource: https://worldpopulationreview.com/static/states/abbr-name.csv

In [59]:
# Open csv state abbrv to state name
sn = pd.read_csv("https://worldpopulationreview.com/static/states/abbr-name.csv", header = None)

# Rename column 
sn = sn.rename(columns = {0: "addr_state" , 1:"length_as"})

# Merge
df = df.merge(sn, how = 'left', on = 'addr_state')

In [60]:
df.loc[df['length_as'] == 'District Of Columbia', 'length_as'] = 'District of Columbia'

## IV. Open GDP information and process for easy joining

In [61]:
# gdp open
gdp = pd.read_csv("SQGDP1__ALL_AREAS_2005_2020.csv")

# get only real GDP description
gdp = gdp.loc[gdp['Description'] == 'Real GDP (millions of chained 2012 dollars)']

# Drop unnecessary columns
gdp = gdp.drop(columns = ['GeoFIPS', 'Region', 'TableName', 'LineCode', 'IndustryClassification', 'Description', 'Unit'])

# Readjust into time-series table
gdp = gdp.melt(id_vars = ['GeoName'],
         var_name = "YEAR:Q",
         value_name = "RealGDP")

# Change time period into datetime pandas format
gdp['YEAR:Q'] = gdp['YEAR:Q'].str.replace(r'(\d+):(Q\d)', r'\1-\2')
gdp['startQuarter'] = pd.to_datetime(gdp['YEAR:Q'])
gdp['endQuarter'] = pd.to_datetime(gdp['startQuarter'] + pd.tseries.offsets.QuarterEnd(0))

# Sort values
gdp = gdp.sort_values(by = ['GeoName', 'YEAR:Q'])

# Get first difference percentage change
gdp['Diff'] = gdp.groupby(['GeoName'])['RealGDP'].pct_change().fillna(0)

# Get previous quarter in 2 new columns
gdp['prevQuarterStartDate'] = gdp['startQuarter'] - pd.tseries.offsets.DateOffset(months = 3)
gdp['prevQuarterEndDate'] = gdp['endQuarter'] - pd.tseries.offsets.DateOffset(months = 3)
gdp['prevQuarterDiff'] = gdp.groupby(['GeoName'])['Diff'].shift(1)
gdp['issue_q'] = pd.PeriodIndex(pd.to_datetime(gdp['startQuarter']), freq = 'Q')
gdp['pqissue_q'] = pd.PeriodIndex(pd.to_datetime(gdp['prevQuarterStartDate']), freq = 'Q')

# Check for final and ready to join
gdp = gdp.rename(columns = {'GeoName':"length_as"})
gdp = gdp[['length_as', 'issue_q', 'Diff', 'pqissue_q', 'prevQuarterDiff']]
gdp

,length_as,issue_q,Diff,pqissue_q,prevQuarterDiff
1,Alabama,2005Q1,0.000000,2004Q4,NaN
61,Alabama,2005Q2,0.011249,2005Q1,0.000000
121,Alabama,2005Q3,-0.000443,2005Q2,0.011249
181,Alabama,2005Q4,0.012301,2005Q3,-0.000443
241,Alabama,2006Q1,0.001972,2005Q4,0.012301
...,...,...,...,...,...
3411,Wyoming,2019Q1,0.013632,2018Q4,0.008723
3471,Wyoming,2019Q2,0.010326,2019Q1,0.013632
3531,Wyoming,2019Q3,0.003315,2019Q2,0.010326
3591,Wyoming,2019Q4,0.000020,2019Q3,0.003315


## V. Join with GDP information

In [62]:
df = pd.merge(df, gdp, on = ['length_as', 'issue_q'], how = 'left')
df = df.drop(columns = 'pqissue_q_x')
df = df.rename(columns = {'pqissue_q_y':'pqissue_q'})
df

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,pqissue_d,issue_q,NAR,log_annual_inc,log_revol_bal,ageOfCredit,length_as,Diff,pqissue_q,prevQuarterDiff
0,NaN,NaN,2500,2500,2500.0,36 months,13.56,84.92,C,C1,...,2018-09-01,2018Q4,-0.889071,4.740371,3.637690,17.667714,New York,-0.006573,2018Q3,0.004246
1,NaN,NaN,30000,30000,30000.0,60 months,18.94,777.23,D,D2,...,2018-09-01,2018Q4,-0.982274,4.954247,4.090470,31.502358,Louisiana,0.002888,2018Q3,0.004128
2,NaN,NaN,5000,5000,5000.0,36 months,17.97,180.69,D,D1,...,2018-09-01,2018Q4,-0.870187,4.772916,3.662758,7.668878,Michigan,-0.002725,2018Q3,0.004887
3,NaN,NaN,4000,4000,4000.0,36 months,18.94,146.51,D,D2,...,2018-09-01,2018Q4,-0.866056,4.963793,3.737908,12.829832,Washington,0.003031,2018Q3,0.014669
4,NaN,NaN,30000,30000,30000.0,60 months,16.14,731.78,C,C4,...,2018-09-01,2018Q4,-1.000990,4.757783,2.919078,17.999001,Maryland,0.002326,2018Q3,0.001665
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2260663,NaN,NaN,12000,12000,12000.0,60 months,14.08,279.72,C,C3,...,2017-07-01,2017Q4,0.657243,4.763435,3.981954,13.749769,Vermont,0.001888,2017Q3,0.005443
2260664,NaN,NaN,12000,12000,12000.0,60 months,25.82,358.01,E,E4,...,2017-07-01,2017Q4,0.310692,4.477136,3.543820,14.587569,Oregon,0.015265,2017Q3,0.007379
2260665,NaN,NaN,10000,10000,10000.0,36 months,11.99,332.10,B,B5,...,2017-07-01,2017Q4,0.516426,4.806187,3.850952,22.664394,Illinois,0.003432,2017Q3,0.008233
2260666,NaN,NaN,12000,12000,12000.0,60 months,21.45,327.69,D,D5,...,2017-07-01,2017Q4,0.681046,4.778158,4.111531,14.253544,Alaska,-0.008250,2017Q3,0.011200


In [63]:
df.shape

(2260668, 156)

In [64]:
df.isnull().sum()

id                 2260668
member_id          2260668
loan_amnt                0
funded_amnt              0
funded_amnt_inv          0
                    ...   
ageOfCredit             29
length_as                0
Diff                     0
pqissue_q                0
prevQuarterDiff          0
Length: 156, dtype: int64

## VI. Open Unemployment Rate information and process for easy joining

In [65]:
bls = pd.read_excel('ststdsadata.xlsx')

# Renaming the state to it's abbreviation 
bls.loc[bls['Unnamed: 1'] == "Alabama", ['Unnamed: 1']] =  'AL'
bls.loc[bls['Unnamed: 1'] == "Alaska", ['Unnamed: 1']] =  'AK'
bls.loc[bls['Unnamed: 1'] == "Arizona", ['Unnamed: 1']] =  'AZ'
bls.loc[bls['Unnamed: 1'] == "Arkansas", ['Unnamed: 1']] =  'AR'
bls.loc[bls['Unnamed: 1'] == "California", ['Unnamed: 1']] =  'CA'
bls.loc[bls['Unnamed: 1'] == "Colorado", ['Unnamed: 1']] =  'CO'
bls.loc[bls['Unnamed: 1'] == "Connecticut", ['Unnamed: 1']] =  'CT'
bls.loc[bls['Unnamed: 1'] == "Delaware", ['Unnamed: 1']] =  'DE'
bls.loc[bls['Unnamed: 1'] == "District of Columbia", ['Unnamed: 1']] =  'DC'
bls.loc[bls['Unnamed: 1'] == "Florida", ['Unnamed: 1']] =  'FL'
bls.loc[bls['Unnamed: 1'] == "Georgia", ['Unnamed: 1']] =  'GA'
bls.loc[bls['Unnamed: 1'] == "Hawaii", ['Unnamed: 1']] =  'HI'
bls.loc[bls['Unnamed: 1'] == "Idaho", ['Unnamed: 1']] =  'ID'
bls.loc[bls['Unnamed: 1'] == "Illinois", ['Unnamed: 1']] =  'IL'
bls.loc[bls['Unnamed: 1'] == "Indiana", ['Unnamed: 1']] =  'IN'
bls.loc[bls['Unnamed: 1'] == "Iowa", ['Unnamed: 1']] =  'IA'
bls.loc[bls['Unnamed: 1'] == "Kansas", ['Unnamed: 1']] =  'KS'
bls.loc[bls['Unnamed: 1'] == "Kentucky", ['Unnamed: 1']] =  'KY'
bls.loc[bls['Unnamed: 1'] == "Louisiana", ['Unnamed: 1']] =  'LA'
bls.loc[bls['Unnamed: 1'] == "Maine", ['Unnamed: 1']] =  'ME'
bls.loc[bls['Unnamed: 1'] == "Maryland", ['Unnamed: 1']] =  'MD'
bls.loc[bls['Unnamed: 1'] == "Massachusetts", ['Unnamed: 1']] =  'MA'
bls.loc[bls['Unnamed: 1'] == "Michigan", ['Unnamed: 1']] =  'MI'
bls.loc[bls['Unnamed: 1'] == "Minnesota", ['Unnamed: 1']] =  'MN'
bls.loc[bls['Unnamed: 1'] == "Mississippi", ['Unnamed: 1']] =  'MS'
bls.loc[bls['Unnamed: 1'] == "Missouri", ['Unnamed: 1']] =  'MO'
bls.loc[bls['Unnamed: 1'] == "Montana", ['Unnamed: 1']] =  'MT'
bls.loc[bls['Unnamed: 1'] == "Nebraska", ['Unnamed: 1']] =  'NE'
bls.loc[bls['Unnamed: 1'] == "Nevada", ['Unnamed: 1']] =  'NV'
bls.loc[bls['Unnamed: 1'] == "New Hampshire", ['Unnamed: 1']] =  'NH'
bls.loc[bls['Unnamed: 1'] == "New Jersey", ['Unnamed: 1']] =  'NJ'
bls.loc[bls['Unnamed: 1'] == "New Mexico", ['Unnamed: 1']] =  'NM'
bls.loc[bls['Unnamed: 1'] == "New York", ['Unnamed: 1']] =  'NY'
bls.loc[bls['Unnamed: 1'] == "North Carolina", ['Unnamed: 1']] =  'NC'
bls.loc[bls['Unnamed: 1'] == "North Dakota", ['Unnamed: 1']] =  'ND'
bls.loc[bls['Unnamed: 1'] == "Ohio", ['Unnamed: 1']] =  'OH'
bls.loc[bls['Unnamed: 1'] == "Oklahoma", ['Unnamed: 1']] =  'OK'
bls.loc[bls['Unnamed: 1'] == "Oregon", ['Unnamed: 1']] =  'OR'
bls.loc[bls['Unnamed: 1'] == "Pennsylvania", ['Unnamed: 1']] =  'PA'
bls.loc[bls['Unnamed: 1'] == "Rhode Island", ['Unnamed: 1']] =  'RI'
bls.loc[bls['Unnamed: 1'] == "South Carolina", ['Unnamed: 1']] =  'SC'
bls.loc[bls['Unnamed: 1'] == "South Dakota", ['Unnamed: 1']] =  'SD'
bls.loc[bls['Unnamed: 1'] == "Tennessee", ['Unnamed: 1']] =  'TN'
bls.loc[bls['Unnamed: 1'] == "Texas", ['Unnamed: 1']] =  'TX'
bls.loc[bls['Unnamed: 1'] == "Utah", ['Unnamed: 1']] =  'UT'
bls.loc[bls['Unnamed: 1'] == "Vermont", ['Unnamed: 1']] =  'VT'
bls.loc[bls['Unnamed: 1'] == "Virginia", ['Unnamed: 1']] =  'VA'
bls.loc[bls['Unnamed: 1'] == "Washington", ['Unnamed: 1']] =  'WA'
bls.loc[bls['Unnamed: 1'] == "West Virginia", ['Unnamed: 1']] =  'WV'
bls.loc[bls['Unnamed: 1'] == "Wisconsin", ['Unnamed: 1']] =  'WI'
bls.loc[bls['Unnamed: 1'] == "Wyoming", ['Unnamed: 1']] =  'WY'

# Creating Month column 
df['Month'] = pd.DatetimeIndex(df['issue_d']).month

# Converting to leading 0, because that's the format used in BLS dataset 
# for instance, January is 01 instead of 1
df["Month"] = df.Month.map("{:02}".format)

# Creating Year column in df 
df['Year'] = pd.DatetimeIndex(df['issue_d']).year

# renaming columns in bls dataset 
bls = bls.rename(columns = {"Unnamed: 1": "addr_state"})
bls = bls.rename(columns = {"Unnamed: 2": "Year"})
bls = bls.rename(columns = {"Unnamed: 3": "Month"})
bls = bls.rename(columns = {"Unnamed: 10": "UR"}) #unemployment rate

bls = pd.DataFrame(bls, columns=['addr_state', 'Year', 'Month', 'UR'])
bls = bls.dropna() #CLEANED bls 
bls.head()

,addr_state,Year,Month,UR
7,AL,1976,01,6.7
8,AK,1976,01,7.1
9,AZ,1976,01,10.3
10,AR,1976,01,7.4
11,CA,1976,01,9.1


In [66]:
bls['YearMonth'] = bls['Year'].astype(str)+'-'+bls['Month'].astype(str)
bls['YearQuarter'] = pd.PeriodIndex(pd.to_datetime(bls['YearMonth']), freq = 'Q')
bls = bls.sort_values(by = ['addr_state', 'YearMonth'])
bls['URDiff'] = bls.groupby(['addr_state'])['UR'].pct_change().fillna(0)
bls['URprevMonthDiff'] = bls.groupby(['addr_state'])['URDiff'].shift(1)
bls


,addr_state,Year,Month,UR,YearMonth,YearQuarter,URDiff,URprevMonthDiff
8,AK,1976,01,7.1,1976-01,1976Q1,0.000000,NaN
61,AK,1976,02,7.1,1976-02,1976Q1,0.000000,0.000000
114,AK,1976,03,7,1976-03,1976Q1,-0.014085,0.000000
167,AK,1976,04,7,1976-04,1976Q2,0.000000,-0.014085
220,AK,1976,05,7,1976-05,1976Q2,0.000000,0.000000
...,...,...,...,...,...,...,...,...
28043,WY,2020,01,3.7,2020-01,2020Q1,0.000000,0.000000
28096,WY,2020,02,3.7,2020-02,2020Q1,0.000000,0.000000
28149,WY,2020,03,3.8,2020-03,2020Q1,0.027027,0.000000
28202,WY,2020,04,9.6,2020-04,2020Q2,1.526316,0.027027


## VII. Join with Unemployment Rate Information

In [67]:
bls['Year']=bls['Year'].astype(int) #has to convert to allow merging; datatypes have to be the same. 
df = pd.merge(df, bls, on = ['Month', 'Year', 'addr_state'], how = 'left')

In [68]:
df.head(5)

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,Diff,pqissue_q,prevQuarterDiff,Month,Year,UR,YearMonth,YearQuarter,URDiff,URprevMonthDiff
0,NaN,NaN,2500,2500,2500.0,36 months,13.56,84.92,C,C1,...,-0.006573,2018Q3,0.004246,12,2018,4,2018-12,2018Q4,0.000000,0.025641
1,NaN,NaN,30000,30000,30000.0,60 months,18.94,777.23,D,D2,...,0.002888,2018Q3,0.004128,12,2018,4.7,2018-12,2018Q4,-0.020833,0.000000
2,NaN,NaN,5000,5000,5000.0,36 months,17.97,180.69,D,D1,...,-0.002725,2018Q3,0.004887,12,2018,4.1,2018-12,2018Q4,0.025000,0.000000
3,NaN,NaN,4000,4000,4000.0,36 months,18.94,146.51,D,D2,...,0.003031,2018Q3,0.014669,12,2018,4.5,2018-12,2018Q4,0.000000,0.022727
4,NaN,NaN,30000,30000,30000.0,60 months,16.14,731.78,C,C4,...,0.002326,2018Q3,0.001665,12,2018,3.8,2018-12,2018Q4,0.000000,0.000000


There are 18 additional columns gained from adding columns and data integration:
1. 'earliest_cr_line_year' - getting the year out of earliest credit line
2. 'pqissue_d' - issue date minus one quarter
3. 'issue_q' - the quarter of issue quarter
4. 'NAR' - Net Annualized Return
5. 'log_annual_inc' - logarithmic of annual inc base 10
6. 'log_revol_bal' - log base 10 of revolving balance
7. 'ageOfCredit' - age of credit in months
8. 'length_as' - unabbreviated state names
9. 'Diff' - GDP growth last quarter to this quarter
10. 'pqissue_q' - previous quarter of issue quarter
11. 'prevQuarterDiff' - GDP growth of two quarters ago to last quarter
12. 'Month' - Month of issue date
13. 'Year' - Year of issue date
14. 'UR' - Unemployment rate during issue date
15. 'YearMonth' - Year month
16. 'YearQuarter' - Year quarter
17. 'URDiff' - Unemployment rate growth from last month to this month
18. 'URprevMonthDiff' - Unemployment rate growth from 2 months ago to last month

In [69]:
df.shape

(2260668, 163)

In [70]:
df.columns.values.tolist()

['id',
 'member_id',
 'loan_amnt',
 'funded_amnt',
 'funded_amnt_inv',
 'term',
 'int_rate',
 'installment',
 'grade',
 'sub_grade',
 'emp_title',
 'emp_length',
 'home_ownership',
 'annual_inc',
 'verification_status',
 'issue_d',
 'loan_status',
 'pymnt_plan',
 'url',
 'desc',
 'purpose',
 'title',
 'zip_code',
 'addr_state',
 'dti',
 'delinq_2yrs',
 'earliest_cr_line',
 'inq_last_6mths',
 'mths_since_last_delinq',
 'mths_since_last_record',
 'open_acc',
 'pub_rec',
 'revol_bal',
 'revol_util',
 'total_acc',
 'initial_list_status',
 'out_prncp',
 'out_prncp_inv',
 'total_pymnt',
 'total_pymnt_inv',
 'total_rec_prncp',
 'total_rec_int',
 'total_rec_late_fee',
 'recoveries',
 'collection_recovery_fee',
 'last_pymnt_d',
 'last_pymnt_amnt',
 'next_pymnt_d',
 'last_credit_pull_d',
 'collections_12_mths_ex_med',
 'mths_since_last_major_derog',
 'policy_code',
 'application_type',
 'annual_inc_joint',
 'dti_joint',
 'verification_status_joint',
 'acc_now_delinq',
 'tot_coll_amt',
 'tot_cur_

# Data Description

In [71]:
# Examine the first 3 rows of the data
df.head(3)

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,Diff,pqissue_q,prevQuarterDiff,Month,Year,UR,YearMonth,YearQuarter,URDiff,URprevMonthDiff
0,NaN,NaN,2500,2500,2500.0,36 months,13.56,84.92,C,C1,...,-0.006573,2018Q3,0.004246,12,2018,4,2018-12,2018Q4,0.000000,0.025641
1,NaN,NaN,30000,30000,30000.0,60 months,18.94,777.23,D,D2,...,0.002888,2018Q3,0.004128,12,2018,4.7,2018-12,2018Q4,-0.020833,0.000000
2,NaN,NaN,5000,5000,5000.0,36 months,17.97,180.69,D,D1,...,-0.002725,2018Q3,0.004887,12,2018,4.1,2018-12,2018Q4,0.025000,0.000000


In [72]:
# Examine the data types of each row in the data frame
df.dtypes

id                       float64
member_id                float64
loan_amnt                  int64
funded_amnt                int64
funded_amnt_inv          float64
                       ...      
UR                        object
YearMonth                 object
YearQuarter        period[Q-DEC]
URDiff                   float64
URprevMonthDiff          float64
Length: 163, dtype: object

In [73]:
# Checking the dimensionality of the data
df.shape

(2260668, 163)

In [74]:
# Describe the data
df.describe()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,int_rate,installment,emp_length,annual_inc,url,...,earliest_cr_line_year,NAR,log_annual_inc,log_revol_bal,ageOfCredit,Diff,prevQuarterDiff,Year,URDiff,URprevMonthDiff
count,0.0,0.0,2.260668e+06,2.260668e+06,2.260668e+06,2.260668e+06,2.260668e+06,2.113761e+06,2.260664e+06,0.0,...,2.260668e+06,2.258247e+06,2.260664e+06,2.260668e+06,2.260639e+06,2.260668e+06,2.260668e+06,2.260668e+06,2.260668e+06,2.260668e+06
mean,NaN,NaN,1.504693e+04,1.504166e+04,1.502344e+04,1.309291e+01,4.458076e+02,5.931359e+00,7.799243e+04,NaN,...,1.999439e+03,inf,4.816103e+00,3.993993e+00,1.639495e+01,5.804401e-03,5.963072e-03,2.015858e+03,-8.777189e-03,-9.101832e-03
std,NaN,NaN,9.190245e+03,9.188413e+03,9.192332e+03,4.832114e+00,2.671737e+02,3.720212e+00,1.126962e+05,NaN,...,1.062342e+01,NaN,2.756264e-01,5.510375e-01,7.679878e+00,7.457629e-03,7.574717e-03,1.792634e+00,1.367873e-02,1.345716e-02
min,NaN,NaN,5.000000e+02,5.000000e+02,0.000000e+00,5.310000e+00,4.930000e+00,0.000000e+00,0.000000e+00,NaN,...,0.000000e+00,-1.000000e+02,0.000000e+00,0.000000e+00,5.037749e-01,-6.971984e-02,-6.971984e-02,2.007000e+03,-5.769231e-02,-5.769231e-02
25%,NaN,NaN,8.000000e+03,8.000000e+03,8.000000e+03,9.490000e+00,2.516500e+02,2.000000e+00,4.600000e+04,NaN,...,1.995000e+03,-2.320214e-01,4.662767e+00,3.774590e+00,1.125280e+01,1.478082e-03,1.602764e-03,2.015000e+03,-2.000000e-02,-2.000000e-02
50%,NaN,NaN,1.290000e+04,1.287500e+04,1.280000e+04,1.262000e+01,3.779900e+02,6.000000e+00,6.500000e+04,NaN,...,2.001000e+03,-2.372438e-02,4.812920e+00,4.054038e+00,1.483398e+01,5.814197e-03,5.952554e-03,2.016000e+03,0.000000e+00,-8.403361e-03
75%,NaN,NaN,2.000000e+04,2.000000e+04,2.000000e+04,1.599000e+01,5.933200e+02,1.000000e+01,9.300000e+04,NaN,...,2.005000e+03,4.896736e-02,4.968488e+00,4.306361e+00,2.025230e+01,1.038282e-02,1.050397e-02,2.017000e+03,0.000000e+00,0.000000e+00
max,NaN,NaN,4.000000e+04,4.000000e+04,4.000000e+04,3.099000e+01,1.719830e+03,1.000000e+01,1.100000e+08,NaN,...,2.015000e+03,inf,8.041393e+00,6.463122e+00,8.325428e+01,9.254971e-02,9.254971e-02,2.018000e+03,5.666667e-01,1.515152e-01


In [75]:
# Find the amount of isnull
na = df.isnull().sum()
print(na)

id                 2260668
member_id          2260668
loan_amnt                0
funded_amnt              0
funded_amnt_inv          0
                    ...   
UR                       0
YearMonth                0
YearQuarter              0
URDiff                   0
URprevMonthDiff          0
Length: 163, dtype: int64


In [76]:
df.to_csv("df_integrated.csv")